In [2]:
%%time
import numpy as np
import pandas as pd
import csv
import time
# %load_ext line_profiler

Wall time: 0 ns


In [3]:
%%time
####
# inputDf = pd.read_excel('inputTestData.xlsx',index_col=0)
caData = pd.read_csv('excel/input/priceData/CA_DATA_11.csv')
priceJanJune17 = pd.read_csv('excel/input/priceData/Price Data Jan to Jun 17.csv')
priceJulDec17 = pd.read_csv('excel/input/priceData/Price Data Jul to Dec 17.csv')

# replace null values of BC_TO_DATE <-- BSE_NSE_EX_DATE
caData = caData.fillna({
    "BC_TO_DATE":caData["BSE_NSE_EX_DATE"]
})
caData['BC_TO_DATE'] = pd.to_datetime(caData['BC_TO_DATE'],format='%d/%m/%Y')
price = pd.DataFrame(priceJanJune17.append(priceJulDec17)).reset_index().drop(columns="index")

FileNotFoundError: [Errno 2] File b'excel/input/priceData/CA_DATA_11.csv' does not exist: b'excel/input/priceData/CA_DATA_11.csv'

In [3]:
%%time
####
price['ISIN'] = price['ISIN'].fillna(0)
price['MTMDATE'] = pd.to_datetime(price['MTMDATE'],format='%d-%m-%Y')
distPrice = price.drop_duplicates(subset=["ISIN","MTMDATE"]).reset_index().drop(columns='index')

dateDf = pd.DataFrame(data = {"Date": pd.date_range(price['MTMDATE'].min(),price['MTMDATE'].max())})
dateDistPrice = pd.merge(dateDf,distPrice,left_on="Date",right_on=["MTMDATE"],how="left")

Wall time: 4.37 s


In [4]:
%%time
####
cols = dateDistPrice['ISIN'].unique()
cols = cols[~pd.isna(cols)]

pivotDf = dateDistPrice.pivot_table(index =['Date'],columns=['ISIN']).fillna(method='ffill').fillna(method='bfill')
# pivotDf = dateDistPrice.pivot_table(index =['Date'],columns=['ISIN'])

pivotDf.columns=pivotDf.columns.droplevel(0)
pivotDf

Wall time: 1.11 s


In [5]:
%%time
####
pivotDf = pivotDf.rename_axis(None, axis=1)
pivotDf

####
pivotDf = pivotDf[cols]
pivotDf

Wall time: 29.9 ms


In [6]:
%%time
####
pivotDf = pivotDf.rename_axis(None, axis=1).stack().reset_index().rename(columns={'level_1':"ISIN",0:"MTMPRICE"})

distCaData = caData.drop_duplicates(subset=["ISIN","BC_TO_DATE"]).reset_index().drop(columns='index')

Wall time: 438 ms


In [7]:
%%time
mergeDF = pd.merge(pivotDf,distCaData,left_on=["ISIN","Date"],right_on=["ISIN","BC_TO_DATE"],how='left')
mergeDF

Wall time: 3.36 s


In [8]:
%%time
####
mergeDF['OLD_RATIO'].fillna(1,inplace=True)
mergeDF['NEW_RATIO'].fillna(1,inplace=True)
finalDf = pd.DataFrame()
finalDf['Date'] = mergeDF['Date']
finalDf['ISIN'] =mergeDF['ISIN']
finalDf['close'] = mergeDF['MTMPRICE']
finalDf['dividend'] = mergeDF['DIVIDEND_RATE'].fillna(0)
# finalDf['dividend'] = finalDf['dividend'].fillna(0)
finalDf['split_ratio'] = (mergeDF['OLD_RATIO']/mergeDF['NEW_RATIO']).fillna(1)
finalDf.set_index('Date',inplace=True)
finalDf.sort_index(inplace=True)

Wall time: 323 ms


In [9]:
%%time
####
rawPrices = finalDf.pivot_table(index='Date',columns='ISIN',values='close')[cols]
dividend = finalDf.pivot_table(index='Date',columns='ISIN',values='dividend')[cols]
splitRatio = finalDf.pivot_table(index='Date',columns='ISIN',values='split_ratio')[cols]

Wall time: 9.47 s


In [10]:
####
# rawPrices.to_excel('excel/output/rawPrices.xlsx')
# dividend.to_excel('excel/output/dividend.xlsx')
# splitRatio.to_excel('excel/output/splitRatio.xlsx')

In [11]:
# %%time
####
def calcAdjustedPrices(rawPrices,dividend,splitRatio):
    rawPrices.sort_index(ascending=False, inplace=True)
    dividend.sort_index(ascending=False, inplace=True)
    splitRatio.sort_index(ascending=False, inplace=True)

    adj_price_df = pd.DataFrame(index=rawPrices.index,data = np.zeros(rawPrices.shape),columns=rawPrices.columns)
    adj_price_df.iloc[0] = round(rawPrices.iloc[0],4)

# simplified formula
    for i in range(1,len(rawPrices)):
        adj_price_df.iloc[i] = (adj_price_df.iloc[i - 1].values *
                       (((rawPrices.iloc[i].values * splitRatio.iloc[i - 1].values) - dividend.iloc[i - 1].values) / rawPrices.iloc[i - 1].values))
    
#     pivotAdjPrice = adj_price_df.stack().reset_index().rename(columns={0:'adjPrice'}).set_index('Date')
#     pivotPrices =  rawPrices.stack().reset_index().rename(columns={0:'close'}).set_index('Date')
#     pivotDividend = dividend.stack().reset_index().rename(columns={0:'dividend'}).set_index('Date')
#     pivotSplit =  splitRatio.stack().reset_index().rename(columns={0:'splitRatio'}).set_index('Date')
    
#     pivotAdjPrice['close'] = pivotPrices['close']
#     pivotAdjPrice['dividend'] = pivotDividend['dividend']
#     pivotAdjPrice['splitRatio'] = pivotSplit['splitRatio']
    
#     pivotAdjPrice.sort_index(ascending=True, inplace=True)

    # pivotAdjPrice[pivotAdjPrice['ISIN']=="INE848E01016"]

    return adj_price_df

In [26]:
rawPrices.shape

(365, 5106)

In [33]:
# %
####
# %lprun -f calcAdjustedPrices calcAdjustedPrices(rawPrices,dividend,splitRatio)
adjPriceDf = calcAdjustedPrices(rawPrices,dividend,splitRatio)

In [ ]:
####
adjPriceDf.to_excel('excel/output/adjustedPrices.xlsx')

In [14]:
%%time
####
testDf2 = pd.DataFrame(data = {'adj' : adjPriceDf['INE042A01014'],'raw':rawPrices['INE042A01014'],'dividend':dividend['INE042A01014']})
with pd.option_context('display.max_rows',None):
#     print((newOutputFinalDf[newOutputFinalDf['ISIN']=="INE660A01013"]))
    print(testDf2.loc['2017-10-21':])

                   adj     raw  dividend
Date                                    
2017-10-21  733.250000  733.25       0.0
2017-10-20  733.250000  733.25       0.0
2017-10-19  733.250000  733.25       0.0
2017-10-18  733.250000  733.25       0.0
2017-10-17  727.400000  727.40       0.0
2017-10-16  722.150000  722.15       0.0
2017-10-15  710.350000  710.35       0.0
2017-10-14  710.350000  710.35       0.0
2017-10-13  710.350000  710.35       0.0
2017-10-12  700.500000  700.50       0.0
2017-10-11  691.200000  691.20       0.0
2017-10-10  698.100000  698.10       0.0
2017-10-09  694.550000  694.55       0.0
2017-10-08  697.200000  697.20       0.0
2017-10-07  697.200000  697.20       0.0
2017-10-06  697.200000  697.20       0.0
2017-10-05  665.650000  665.65       0.0
2017-10-04  656.750000  656.75       0.0
2017-10-03  658.750000  658.75       0.0
2017-10-02  659.650000  659.65       0.0
2017-10-01  659.650000  659.65       0.0
2017-09-30  659.650000  659.65       0.0
2017-09-29  659.

In [ ]:
# adjustedPrices = rawPrices.copy()
# adjustedPrices = pd.DataFrame(data = {'adj':adjustedPrices['INE042A01014'],'raw':rawPrices['INE042A01014'],'dividend':dividend['INE042A01014']})
# adjustedPrices['INE042A01014'] = round(adjustedPrices['INE042A01014'].shift(1)*((rawPrices['INE042A01014']-dividend['INE042A01014'].shift(1))/rawPrices['INE042A01014'].shift(1)),4)
# adjustedPrices = round(adjustedPrices.shift(-1)*(((rawPrices*splitRatio.shift(-1))-dividend.shift(-1))/rawPrices.shift(-1)),4)

# adjustedPrices['adj'] = adjustedPrices['raw']

# adjustedPrices['adj'] = adjustedPrices['adj'].where(adjustedPrices['dividend']>0,adjustedPrices['adj'].shift(1)*((adjustedPrices['raw']-adjustedPrices['dividend'].shift(1))/adjustedPrices['raw'].shift(1)))

# adjustedPrices['adj'] = adjustedPrices['adj'].where(adjustedPrices['dividend']>0,(adjustedPrices['adj'].shift(1)+adjustedPrices['dividend']))

# testDf = pd.DataFrame(data = {'adj':adjustedPrices['INE042A01014'],'raw':rawPrices['INE042A01014'],'dividend':dividend['INE042A01014']})

# with pd.option_context('display.max_rows',None):
# #     print((newOutputFinalDf[newOutputFinalDf['ISIN']=="INE660A01013"]))
#     print(adjustedPrices.loc['2017-10-21':])

adjustedPrices['INE042A01014']==adjPriceDf['INE042A01014']

(adjustedPrices['INE042A01014']==outputFinalDf['adj_close'].sort_index(ascending=False))

(round(rawPrices,4)==adjPriceDf).sum()

(adjustedPrices == adjPriceDf).sum()

adjPriceDf['US92657J1016']

In [ ]:
def calculate_adjusted_prices(df, column):
    """ Vectorized approach for calculating the adjusted prices for the
    specified column in the provided DataFrame. This creates a new column
    called 'adj_<column name>' with the adjusted prices. This function requires
    that the DataFrame have columns with dividend and split_ratio values.

    :param df: DataFrame with raw prices along with dividend and split_ratio
        values
    :param column: String of which price column should have adjusted prices
        created for it
    :return: DataFrame with the addition of the adjusted price column
    """
    adj_column = 'adj_' + column

    # Reverse the DataFrame order, sorting by date in descending order
    df.sort_index(ascending=False, inplace=True)

    price_col = df[column].values
    split_col = df['split_ratio'].values
    dividend_col = df['dividend'].values
    adj_price_col = np.zeros(len(df.index))
    adj_price_col[0] = price_col[0]

    for i in range(1, len(price_col)):
        adj_price_col[i] = round((adj_price_col[i - 1] + adj_price_col[i - 1] *
                   (((price_col[i] * split_col[i - 1]) -
                     price_col[i - 1] -
                     dividend_col[i - 1]) / price_col[i - 1])), 4)

    df[adj_column] = adj_price_col

    # Change the DataFrame order back to dates ascending
    df.sort_index(ascending=True, inplace=True)

    return df

In [ ]:
start2 = time.time()
outputFinalDf = calculate_adjusted_prices(finalDf[finalDf['ISIN']=="INE042A01014"],"close")
time.time()-start2

### PCT CHANGE / DATE / ROLLING PCT CHANGE

In [ ]:
outputFinalDf.asfreq('M')['adj_close'].pct_change()

In [ ]:
adjPriceDf1 = adjPriceDf.reset_index()

In [ ]:
adjPriceDf1['Date'].asfreq='BM'

In [ ]:
adjPriceDf1.set_index('Date').index

In [ ]:
with pd.option_context('display.max_rows',None):
    print((newOutputFinalDf[newOutputFinalDf['ISIN']=="INE660A01013"]))

In [ ]:
newOutputFinalDf[newOutputFinalDf[newOutputFinalDf['ISIN']=="INE105A01035"]['dividend']!=0]

In [ ]:
start2 = time.time()
outputFinalDf = calculate_adjusted_prices(finalDf,"close")
time.time()-start2